In [23]:
# THIS SCRIPT PREDICTS THE MotionBert COORDINATES FOR EVERY FILE IN THE 3DPW DATASET USING THE ALPHAPOSE COORDINATES AND STORES THERM IN 

In [24]:
import os
import numpy as np
import argparse
from tqdm import tqdm
import imageio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from lib.utils.tools import *
from lib.utils.learning import *
from lib.utils.utils_data import flip_data
from lib.data.dataset_wild import WildDetDataset
from lib.utils.vismo import render_and_save
import os
import glob

In [25]:
class Args:
    def __init__(self):
        # Define all your arguments as class attributes with default values
        self.config = "configs/pose3d/MB_ft_h36m_global_lite.yaml"  # Path to the config file
        self.evaluate = "checkpoint/pose3d/FT_MB_lite_MB_ft_h36m_global_lite/best_epoch.bin"  # Checkpoint to evaluate
        self.json_path = None  # Alphapose detection result JSON path
        self.vid_path = None  # Video path
        self.out_path = None  # Output path
        self.pixel = False  # Align with pixel coordinates
        self.focus = None  # Target person ID
        self.clip_len = 243  # Clip length for network input

# Example usage
def main():
    args = Args()  # Create an instance of Args
    print(f"Config: {args.config}")
    print(f"Evaluate: {args.evaluate}")
    # Add further usage of the arguments
    # e.g., get_config(args.config)

main()


Config: configs/pose3d/MB_ft_h36m_global_lite.yaml
Evaluate: checkpoint/pose3d/FT_MB_lite_MB_ft_h36m_global_lite/best_epoch.bin


In [26]:
opts = Args()
args = get_config(opts.config)

In [27]:
video_paths = []
json_paths = []
out_paths = []

In [28]:
opts.vid_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles\downtown_arguing_00\downtown_arguing_00.mp4"
opts.json_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\person_1.json"
opts.out_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_arguing_00\MotionBert"

In [29]:

model_backbone = load_backbone(args)
if torch.cuda.is_available():
    model_backbone = nn.DataParallel(model_backbone)
    model_backbone = model_backbone.cuda()

print('Loading checkpoint', opts.evaluate)
checkpoint = torch.load(opts.evaluate, map_location=lambda storage, loc: storage)
model_backbone.load_state_dict(checkpoint['model_pos'], strict=True)
model_pos = model_backbone
model_pos.eval()

Loading checkpoint checkpoint/pose3d/FT_MB_lite_MB_ft_h36m_global_lite/best_epoch.bin


DataParallel(
  (module): DSTformer(
    (joints_embed): Linear(in_features=3, out_features=256, bias=True)
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks_st): ModuleList(
      (0): Block(
        (norm1_s): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (norm1_t): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (attn_s): Attention(
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=256, out_features=256, bias=True)
          (qkv): Linear(in_features=256, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (attn_t): Attention(
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=256, out_features=256, bias=True)
          (qkv): Linear(in_features=256, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2_s): LayerNorm((256,), eps=1e-06, e

In [30]:
def main(opts):

    testloader_params = {
        'batch_size': 1,
        'shuffle': False,
        'num_workers': 8,
        'pin_memory': True,
        'prefetch_factor': 4,
        'persistent_workers': True,
        'drop_last': False
    }

    vid = imageio.get_reader(opts.vid_path, 'ffmpeg')
    fps_in = vid.get_meta_data()['fps']
    vid_size = vid.get_meta_data()['size']
    os.makedirs(opts.out_path, exist_ok=True)

    if opts.pixel:
        wild_dataset = WildDetDataset(opts.json_path, clip_len=opts.clip_len, vid_size=vid_size, scale_range=None, focus=opts.focus)
    else:
        wild_dataset = WildDetDataset(opts.json_path, clip_len=opts.clip_len, scale_range=[1, 1], focus=opts.focus)

    test_loader = DataLoader(wild_dataset, **testloader_params)

    results_all = []
    with torch.no_grad():
        for batch_input in tqdm(test_loader):
            N, T = batch_input.shape[:2]
            if torch.cuda.is_available():
                batch_input = batch_input.cuda()
            if args.no_conf:
                batch_input = batch_input[:, :, :, :2]
            if args.flip:
                batch_input_flip = flip_data(batch_input)
                predicted_3d_pos_1 = model_pos(batch_input)
                predicted_3d_pos_flip = model_pos(batch_input_flip)
                predicted_3d_pos_2 = flip_data(predicted_3d_pos_flip)
                predicted_3d_pos = (predicted_3d_pos_1 + predicted_3d_pos_2) / 2.0
            else:
                predicted_3d_pos = model_pos(batch_input)
            if args.rootrel:
                predicted_3d_pos[:, :, 0, :] = 0
            else:
                predicted_3d_pos[:, 0, 0, 2] = 0
            if args.gt_2d:
                predicted_3d_pos[..., :2] = batch_input[..., :2]
            results_all.append(predicted_3d_pos.cpu().numpy())

    results_all = np.hstack(results_all)
    results_all = np.concatenate(results_all)
    #render_and_save(results_all, f'{opts.out_path}/X3D.mp4', keep_imgs=False, fps=fps_in)
    if opts.pixel:
        results_all = results_all * (min(vid_size) / 2.0)
        results_all[:, :, :2] = results_all[:, :, :2] + np.array(vid_size) / 2.0
    np.save(f'{opts.out_path}/X3D.npy', results_all)

In [31]:
import os
import glob
import gc

# Paths
video_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\imageFiles\imageFiles"
json_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"
output_base_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"

overwrite = True  # Set to True if you want to force overwriting existing files

# Iterate through each folder in the video base path
for folder_name in os.listdir(video_base_path):
    folder_path = os.path.join(video_base_path, folder_name)

    if os.path.isdir(folder_path):
        video_file = os.path.join(folder_path, f"{folder_name}.mp4")

        if os.path.exists(video_file):
            json_folder = os.path.join(json_base_path, folder_name)

            if os.path.isdir(json_folder):
                json_files = glob.glob(os.path.join(json_folder, "person_*.json"))

                for json_file in json_files:
                    person_id = os.path.splitext(os.path.basename(json_file))[0].split('_')[-1]
                    output_path = os.path.join(output_base_path, folder_name, "MotionBert", f"person_{person_id}")

                    x3d_mp4 = os.path.join(output_path, "X3D.mp4")
                    x3d_npy = os.path.join(output_path, "X3D.npy")

                    if overwrite or not (os.path.exists(x3d_mp4) and os.path.exists(x3d_npy)):
                        print(f"Processing folder: {folder_name}, person: {person_id}")
                        
                        os.makedirs(output_path, exist_ok=True)

                        opts.vid_path = video_file
                        opts.json_path = json_file
                        opts.out_path = output_path
                        main(opts)

                        # Free up memory after processing
                    else:
                        print(f"Skipping {output_path}, output files already exist.")


Processing folder: downtown_arguing_00, person: 1


100%|██████████| 4/4 [00:11<00:00,  2.92s/it]


Processing folder: downtown_arguing_00, person: 2


100%|██████████| 4/4 [00:11<00:00,  2.93s/it]


Processing folder: downtown_bar_00, person: 1


...

100%|██████████| 4/4 [00:11<00:00,  2.90s/it]


Processing folder: office_phoneCall_00, person: 2


100%|██████████| 4/4 [00:11<00:00,  2.88s/it]


Processing folder: outdoors_fencing_01, person: 1


100%|██████████| 4/4 [00:11<00:00,  2.93s/it]


In [41]:
import numpy as np
x3d = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bus_00\MotionBert\person_1\X3D.npy")

In [42]:
x3d.shape

(2178, 17, 3)

In [34]:
import json

# Path to the JSON file
file_path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bus_00\person_1.json"

# Open and load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Access data
print(data)  # Prints the loaded JSON data as a Python dictionary or list


[{'image_id': '0.jpg', 'category_id': 1, 'keypoints': [802.8881225585938, 391.0878601074219, 0.9228121638298035, 834.54052734375, 375.26165771484375, 0.9462135434150696, 787.0618896484375, 359.4354553222656, 0.9411659836769104, 897.8453369140625, 391.0878601074219, 0.9296779632568359, 755.4094848632812, 391.0878601074219, 0.8662592172622681, 961.1500854492188, 596.8284301757812, 0.9385396242141724, 692.104736328125, 581.0022583007812, 0.9548125267028809, 976.976318359375, 850.047607421875, 0.92436283826828, 628.7999267578125, 818.395263671875, 0.9089283347129822, 976.976318359375, 1071.6143798828125, 0.9227245450019836, 612.9736938476562, 1024.1358642578125, 0.8676513433456421, 866.1929321289062, 1071.6143798828125, 0.8669221997261047, 692.104736328125, 1055.7882080078125, 0.8319399952888489, 818.7142944335938, 1388.138427734375, 0.8714492917060852, 692.104736328125, 1372.312255859375, 0.9266476631164551, 771.2357177734375, 1657.183837890625, 0.8806560635566711, 707.930908203125, 1673.

In [35]:
len(data)

2178

In [36]:
all_img_ids = []

for frame in data:
    all_img_ids.append(int(frame['image_id'].split('.')[0]))

In [38]:
len(list(set(all_img_ids)))

2178

In [17]:
list(set(all_img_ids))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [6]:
# Generate the complete range of numbers from 0 to 1239
full_range = set(range(0, 1239))

# Convert the given list to a set
given_numbers_set = set(all_img_ids)

# Find the missing numbers
missing_numbers = full_range - given_numbers_set

# Print the missing numbers
print("Missing numbers are:", sorted(missing_numbers))

Missing numbers are: [614, 615]


In [17]:
# Extract all numeric parts of the `image_id` keys, handling unexpected formats
image_ids = []
for item in data:
    try:
        # Split and convert the numeric part after 'jpg.'
        numeric_id = int(item['image_id'].split('.')[1])
        image_ids.append(numeric_id)
    except (IndexError, ValueError):
        print(f"Skipping invalid image_id: {item['image_id']}")

# Sort the extracted IDs
image_ids = sorted(image_ids)

# Generate the full range of IDs
expected_ids = set(range(image_ids[0], image_ids[-1] + 1))

# Find the missing IDs
existing_ids = set(image_ids)
missing_ids = expected_ids - existing_ids

# Output the missing IDs
print(f"Missing image IDs: {sorted(missing_ids)}")


Skipping invalid image_id: 0.jpg
Skipping invalid image_id: 1.jpg
Skipping invalid image_id: 2.jpg
Skipping invalid image_id: 3.jpg
Skipping invalid image_id: 4.jpg
Skipping invalid image_id: 5.jpg
Skipping invalid image_id: 6.jpg
Skipping invalid image_id: 7.jpg
Skipping invalid image_id: 8.jpg
Skipping invalid image_id: 9.jpg
Skipping invalid image_id: 10.jpg
Skipping invalid image_id: 11.jpg
Skipping invalid image_id: 12.jpg
Skipping invalid image_id: 13.jpg
Skipping invalid image_id: 14.jpg
Skipping invalid image_id: 15.jpg
Skipping invalid image_id: 16.jpg
Skipping invalid image_id: 17.jpg
Skipping invalid image_id: 18.jpg
Skipping invalid image_id: 19.jpg
Skipping invalid image_id: 20.jpg
Skipping invalid image_id: 21.jpg
Skipping invalid image_id: 22.jpg
Skipping invalid image_id: 23.jpg
Skipping invalid image_id: 24.jpg
Skipping invalid image_id: 25.jpg
Skipping invalid image_id: 26.jpg
Skipping invalid image_id: 27.jpg
Skipping invalid image_id: 28.jpg
Skipping invalid image_i

IndexError: list index out of range